# Training Deep Neural Nets

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot

In [2]:
import tensorflow as tf

In [3]:
from tensorflow import keras

---

# Xavier and He Initialization

In [4]:
[name for name in dir(keras.initializers) if not name.startswith("_")]

['Constant',
 'GlorotNormal',
 'GlorotUniform',
 'Identity',
 'Initializer',
 'Ones',
 'Orthogonal',
 'RandomNormal',
 'RandomUniform',
 'TruncatedNormal',
 'VarianceScaling',
 'Zeros',
 'constant',
 'deserialize',
 'get',
 'glorot_normal',
 'glorot_uniform',
 'he_normal',
 'he_uniform',
 'identity',
 'lecun_normal',
 'lecun_uniform',
 'ones',
 'orthogonal',
 'serialize',
 'zeros']

In [5]:
# By default, Keras uses Glorot initialization with a uniform distribution. You can change this to He initialization

keras.layers.Dense(10, activation="relu", kernel_initializer="he_normal")

In [6]:
# If you want He initialization with a uniform distribution, but based on fanavg rather than fanin, 
  # you can use the VarianceScaling initializer like this:

init = keras.initializers.VarianceScaling(scale=2., mode='fan_avg',
                                          distribution='uniform')


keras.layers.Dense(10, activation="relu", kernel_initializer=init)

---

# Neural Network with Leaky ReLU

In [8]:
# Working with fashion mnist dataset

(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()

In [9]:
X_train_full = X_train_full / 255.0

In [10]:
X_test = X_test / 255.0

In [11]:
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]

In [12]:
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [13]:
tf.random.set_seed(42)
np.random.seed(42)

In [14]:
# Modelling with Leaky ReLU

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(100, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(10, activation="softmax")
])

In [15]:
# Compiling the Model

model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

In [16]:
# Training the Model

result_leaky_1 = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Train on 55000 samples, validate on 5000 samples
Epoch 1/10
55000/55000 [==============================] - 3s 52us/sample - loss: 1.2810 - accuracy: 0.6205 - val_loss: 0.8869 - val_accuracy: 0.7160
Epoch 2/10
55000/55000 [==============================] - 3s 46us/sample - loss: 0.7952 - accuracy: 0.7369 - val_loss: 0.7132 - val_accuracy: 0.7626
Epoch 3/10
55000/55000 [==============================] - 3s 46us/sample - loss: 0.6817 - accuracy: 0.7726 - val_loss: 0.6385 - val_accuracy: 0.7896
Epoch 4/10
55000/55000 [==============================] - 2s 45us/sample - loss: 0.6219 - accuracy: 0.7941 - val_loss: 0.5931 - val_accuracy: 0.8016
Epoch 5/10
55000/55000 [==============================] - 2s 45us/sample - loss: 0.5829 - accuracy: 0.8074 - val_loss: 0.5607 - val_accuracy: 0.8164
Epoch 6/10
55000/55000 [==============================] - 2s 45us/sample - loss: 0.5552 - accuracy: 0.8173 - val_loss: 0.5354 - val_accuracy: 0.8242
Epoch 7/10
55000/55000 [==============================] -

---

# Working with PReLU

In [17]:
tf.random.set_seed(42)
np.random.seed(42)

In [18]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer="he_normal"),
    keras.layers.PReLU(),
    keras.layers.Dense(100, kernel_initializer="he_normal"),
    keras.layers.PReLU(),
    keras.layers.Dense(10, activation="softmax")
])

In [19]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

In [20]:
result_prelu_1 = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Train on 55000 samples, validate on 5000 samples
Epoch 1/10
55000/55000 [==============================] - 3s 57us/sample - loss: 1.3452 - accuracy: 0.6203 - val_loss: 0.9241 - val_accuracy: 0.7170
Epoch 2/10
55000/55000 [==============================] - 3s 47us/sample - loss: 0.8196 - accuracy: 0.7364 - val_loss: 0.7314 - val_accuracy: 0.7602
Epoch 3/10
55000/55000 [==============================] - 2s 45us/sample - loss: 0.6970 - accuracy: 0.7702 - val_loss: 0.6517 - val_accuracy: 0.7876
Epoch 4/10
55000/55000 [==============================] - 3s 47us/sample - loss: 0.6333 - accuracy: 0.7914 - val_loss: 0.6033 - val_accuracy: 0.8058
Epoch 5/10
55000/55000 [==============================] - 2s 45us/sample - loss: 0.5917 - accuracy: 0.8049 - val_loss: 0.5689 - val_accuracy: 0.8164
Epoch 6/10
55000/55000 [==============================] - 3s 48us/sample - loss: 0.5619 - accuracy: 0.8144 - val_loss: 0.5417 - val_accuracy: 0.8222
Epoch 7/10
55000/55000 [==============================] -

---

Implementing ELU in TensorFlow is trivial, just specify the activation function when building each layer:

In [21]:
keras.layers.Dense(10, activation="elu")

---

# Implementing with SELU|

This activation function was proposed in this great paper by Günter Klambauer, Thomas Unterthiner and Andreas Mayr, published in June 2017. During training, a neural network composed exclusively of a stack of dense layers using the SELU activation function and LeCun initialization will self-normalize: the output of each layer will tend to preserve the same mean and variance during training, which solves the vanishing/exploding gradients problem. As a result, this activation function outperforms the other activation functions very significantly for such neural nets, so you should really try it out. Unfortunately, the self-normalizing property of the SELU activation function is easily broken: you cannot use ℓ1 or ℓ2 regularization, regular dropout, max-norm, skip connections or other non-sequential topologies (so recurrent neural networks won't self-normalize). However, in practice it works quite well with sequential CNNs. If you break self-normalization, SELU will not necessarily outperform other activation functions.

---

By default, the SELU hyperparameters (scale and alpha) are tuned in such a way that the mean output of each neuron remains close to 0, and the standard deviation remains close to 1 (assuming the inputs are standardized with mean 0 and standard deviation 1 too). Using this activation function, even a 1,000 layer deep neural network preserves roughly mean 0 and standard deviation 1 across all layers, avoiding the exploding/vanishing gradients problem:

---

Let's create a neural net for Fashion MNIST with 100 hidden layers, using the SELU activation function:

In [23]:
# With, selu, it's better to use initializer as 'lecun_normal' 

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))
model.add(keras.layers.Dense(300, activation="selu",
                             kernel_initializer="lecun_normal"))
for layer in range(99):
    model.add(keras.layers.Dense(100, activation="selu",
                                 kernel_initializer="lecun_normal"))
model.add(keras.layers.Dense(10, activation="softmax"))

In [24]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

---

Now let's train it. Do not forget to scale the inputs to mean 0 and standard deviation 1

In [25]:
pixel_means = X_train.mean(axis=0, keepdims=True)
pixel_stds = X_train.std(axis=0, keepdims=True)
X_train_scaled = (X_train - pixel_means) / pixel_stds
X_valid_scaled = (X_valid - pixel_means) / pixel_stds
X_test_scaled = (X_test - pixel_means) / pixel_stds

In [26]:
result_selu_1 = model.fit(X_train_scaled, y_train, epochs=5,
                    validation_data=(X_valid_scaled, y_valid))

Train on 55000 samples, validate on 5000 samples
Epoch 1/5
55000/55000 [==============================] - 17s 315us/sample - loss: 1.0601 - accuracy: 0.5950 - val_loss: 0.7134 - val_accuracy: 0.7250
Epoch 2/5
55000/55000 [==============================] - 16s 283us/sample - loss: 0.6712 - accuracy: 0.7531 - val_loss: 0.6003 - val_accuracy: 0.7836
Epoch 3/5
55000/55000 [==============================] - 15s 277us/sample - loss: 0.5871 - accuracy: 0.7837 - val_loss: 0.5562 - val_accuracy: 0.8050
Epoch 4/5
55000/55000 [==============================] - 15s 279us/sample - loss: 0.6312 - accuracy: 0.7733 - val_loss: 0.7417 - val_accuracy: 0.7346
Epoch 5/5
55000/55000 [==============================] - 15s 278us/sample - loss: 0.5742 - accuracy: 0.7987 - val_loss: 0.4937 - val_accuracy: 0.8288


---

#### Now, let's see what would have happen in case of ReLU function Instead.

In [27]:
np.random.seed(42)
tf.random.set_seed(42)

In [28]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))
model.add(keras.layers.Dense(300, activation="relu", kernel_initializer="he_normal"))
for layer in range(99):
    model.add(keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"))
model.add(keras.layers.Dense(10, activation="softmax"))

In [29]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

In [30]:
result_relu_2 = model.fit(X_train_scaled, y_train, epochs=5,
                    validation_data=(X_valid_scaled, y_valid))

Train on 55000 samples, validate on 5000 samples
Epoch 1/5
55000/55000 [==============================] - 16s 288us/sample - loss: 1.7490 - accuracy: 0.2882 - val_loss: 1.3264 - val_accuracy: 0.4158
Epoch 2/5
55000/55000 [==============================] - 14s 260us/sample - loss: 1.1749 - accuracy: 0.5042 - val_loss: 1.4592 - val_accuracy: 0.5054
Epoch 3/5
55000/55000 [==============================] - 14s 257us/sample - loss: 0.9938 - accuracy: 0.5949 - val_loss: 0.9237 - val_accuracy: 0.6486
Epoch 4/5
55000/55000 [==============================] - 14s 254us/sample - loss: 1.0075 - accuracy: 0.5826 - val_loss: 0.8632 - val_accuracy: 0.6590
Epoch 5/5
55000/55000 [==============================] - 14s 253us/sample - loss: 0.8634 - accuracy: 0.6570 - val_loss: 0.7856 - val_accuracy: 0.7016


Not great at all, we suffered from the vanishing/exploding gradients problem.

---

---

# Implementing the Batch Normalization

In [31]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])

In [32]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_4 (Flatten)          (None, 784)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 784)               3136      
_________________________________________________________________
dense_211 (Dense)            (None, 300)               235500    
_________________________________________________________________
batch_normalization_1 (Batch (None, 300)               1200      
_________________________________________________________________
dense_212 (Dense)            (None, 100)               30100     
_________________________________________________________________
batch_normalization_2 (Batch (None, 100)               400       
_________________________________________________________________
dense_213 (Dense)            (None, 10)               

In [33]:
# Compiing the Model

model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

In [34]:
# Training the Model

result_bn_1 = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Train on 55000 samples, validate on 5000 samples
Epoch 1/10
55000/55000 [==============================] - 5s 82us/sample - loss: 0.8761 - accuracy: 0.7122 - val_loss: 0.5509 - val_accuracy: 0.8230
Epoch 2/10
55000/55000 [==============================] - 4s 71us/sample - loss: 0.5737 - accuracy: 0.8037 - val_loss: 0.4722 - val_accuracy: 0.8456
Epoch 3/10
55000/55000 [==============================] - 4s 74us/sample - loss: 0.5142 - accuracy: 0.8229 - val_loss: 0.4376 - val_accuracy: 0.8576
Epoch 4/10
55000/55000 [==============================] - 4s 75us/sample - loss: 0.4826 - accuracy: 0.8327 - val_loss: 0.4134 - val_accuracy: 0.8644
Epoch 5/10
55000/55000 [==============================] - 4s 75us/sample - loss: 0.4570 - accuracy: 0.8413 - val_loss: 0.3989 - val_accuracy: 0.8650
Epoch 6/10
55000/55000 [==============================] - 4s 74us/sample - loss: 0.4432 - accuracy: 0.8454 - val_loss: 0.3870 - val_accuracy: 0.8696
Epoch 7/10
55000/55000 [==============================] -

---

# Gradient Clipping

---

All Keras optimizers accept clipnorm or clipvalue arguments:

In [35]:
optimizer = keras.optimizers.SGD(clipvalue=1.0)

In [36]:
optimizer = keras.optimizers.SGD(clipnorm=1.0)

---

# Reusing the Pretrained Layers

Let's split the fashion MNIST training set in two:

X_train_A: all images of all items except for sandals and shirts (classes 5 and 6).

X_train_B: a much smaller training set of just the first 200 images of sandals or shirts.

The validation set and the test set are also split this way, but without restricting the number of images.

We will train a model on set A (classification task with 8 classes), and try to reuse it to tackle set B (binary classification). We hope to transfer a little bit of knowledge from task A to task B, since classes in set A (sneakers, ankle boots, coats, t-shirts, etc.) are somewhat similar to classes in set B (sandals and shirts). However, since we are using Dense layers, only patterns that occur at the same location can be reused (in contrast, convolutional layers will transfer much better, since learned patterns can be detected anywhere on the image, as we will see in the CNN chapter).

In [37]:
def split_dataset(X, y):
    y_5_or_6 = (y == 5) | (y == 6) # sandals or shirts
    y_A = y[~y_5_or_6]
    y_A[y_A > 6] -= 2 # class indices 7, 8, 9 should be moved to 5, 6, 7
    y_B = (y[y_5_or_6] == 6).astype(np.float32) # binary classification task: is it a shirt (class 6)?
    return ((X[~y_5_or_6], y_A),
            (X[y_5_or_6], y_B))

(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

In [38]:
X_train_A.shape

(43986, 28, 28)

In [39]:
X_train_B.shape

(200, 28, 28)

In [40]:
y_train_A[:30]

array([4, 0, 5, 7, 7, 7, 4, 4, 3, 4, 0, 1, 6, 3, 4, 3, 2, 6, 5, 3, 4, 5,
       1, 3, 4, 2, 0, 6, 7, 1], dtype=uint8)

In [41]:
y_train_A[:30]

array([4, 0, 5, 7, 7, 7, 4, 4, 3, 4, 0, 1, 6, 3, 4, 3, 2, 6, 5, 3, 4, 5,
       1, 3, 4, 2, 0, 6, 7, 1], dtype=uint8)

In [42]:
y_train_B[:30]

array([1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1.], dtype=float32)

In [43]:
tf.random.set_seed(42)
np.random.seed(42)

In [44]:
# model_A created for classifying the 8 class data

model_A = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_A.add(keras.layers.Dense(n_hidden, activation="selu"))
model_A.add(keras.layers.Dense(8, activation="softmax"))     # final layer have 8 categories to be classified

In [45]:
model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=keras.optimizers.SGD(lr=1e-3),
                metrics=["accuracy"])

In [46]:
result_pretrained_1 = model_A.fit(X_train_A, y_train_A, epochs=20,
                    validation_data=(X_valid_A, y_valid_A))

Train on 43986 samples, validate on 4014 samples
Epoch 1/20
43986/43986 [==============================] - 3s 57us/sample - loss: 0.5902 - accuracy: 0.8130 - val_loss: 0.3782 - val_accuracy: 0.8690
Epoch 2/20
43986/43986 [==============================] - 2s 53us/sample - loss: 0.3517 - accuracy: 0.8784 - val_loss: 0.3374 - val_accuracy: 0.8837
Epoch 3/20
43986/43986 [==============================] - 2s 50us/sample - loss: 0.3162 - accuracy: 0.8897 - val_loss: 0.3017 - val_accuracy: 0.8959
Epoch 4/20
43986/43986 [==============================] - 2s 52us/sample - loss: 0.2969 - accuracy: 0.8975 - val_loss: 0.2912 - val_accuracy: 0.9021
Epoch 5/20
43986/43986 [==============================] - 2s 51us/sample - loss: 0.2831 - accuracy: 0.9026 - val_loss: 0.2820 - val_accuracy: 0.9018
Epoch 6/20
43986/43986 [==============================] - 2s 51us/sample - loss: 0.2726 - accuracy: 0.9067 - val_loss: 0.2738 - val_accuracy: 0.9068
Epoch 7/20
43986/43986 [==============================] -

In [47]:
# saving the model

model_A.save("my_model_A.h5")

In [48]:
# creating another model

# Model_B is kind of binary classifier like to guess the positive or negative.
model_B = keras.models.Sequential()
model_B.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_B.add(keras.layers.Dense(n_hidden, activation="selu"))
model_B.add(keras.layers.Dense(1, activation="sigmoid"))

In [49]:
# compiling 

model_B.compile(loss="binary_crossentropy",
                optimizer=keras.optimizers.SGD(lr=1e-3),
                metrics=["accuracy"])

In [50]:
result_pretrained_2 = model_B.fit(X_train_B, y_train_B, epochs=20,
                      validation_data=(X_valid_B, y_valid_B))

Train on 200 samples, validate on 986 samples
Epoch 1/20
200/200 [==============================] - 0s 2ms/sample - loss: 0.9509 - accuracy: 0.4800 - val_loss: 0.6533 - val_accuracy: 0.5568
Epoch 2/20
200/200 [==============================] - 0s 299us/sample - loss: 0.5837 - accuracy: 0.7100 - val_loss: 0.4825 - val_accuracy: 0.8479
Epoch 3/20
200/200 [==============================] - 0s 280us/sample - loss: 0.4527 - accuracy: 0.8750 - val_loss: 0.4097 - val_accuracy: 0.8945
Epoch 4/20
200/200 [==============================] - 0s 280us/sample - loss: 0.3869 - accuracy: 0.9050 - val_loss: 0.3630 - val_accuracy: 0.9209
Epoch 5/20
200/200 [==============================] - 0s 300us/sample - loss: 0.3404 - accuracy: 0.9300 - val_loss: 0.3302 - val_accuracy: 0.9280
Epoch 6/20
200/200 [==============================] - 0s 271us/sample - loss: 0.3073 - accuracy: 0.9350 - val_loss: 0.3026 - val_accuracy: 0.9381
Epoch 7/20
200/200 [==============================] - 0s 270us/sample - loss: 0.

In [51]:
# model_B task is similar to model_A and model_B should give more better result as there is it's just the 
 # problem of binary classifier.
    

# Loading back the model_A
model_A = keras.models.load_model("my_model_A.h5")

In [52]:
# creating new model based on model_A

# using the layers of model A
model_B_on_A = keras.models.Sequential(model_A.layers[:-1])

In [53]:
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

Note that model_A and model_B_on_A now share some layers.

When you train model_B_on_A, it will also affect model_A. If you want to avoid that, you need to clone model_A before you reuse its layers

---

In [54]:
# You must clone model A’s architecture, then copy its weights (since clone_model() does not clone the weights

model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())

---

Now we could just train model_B_on_A for task B, but since the new output layer was initialized randomly, it will make large errors, at least during the first few epochs, so there will be large error gradients that may wreck the reused weights. To avoid this, one approach is to freeze the reused layers during the first few epochs, giving the new layer some time to learn reasonable weights. To do this, simply set every layer’s train able attribute to False and compile the model

In [55]:
# freezing the previous layer for some time

for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

In [56]:
model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(lr=1e-3),
                     metrics=["accuracy"])

---

Next, we can train the model for a few epochs, then unfreeze the reused layers (which requires compiling the model again) and continue training to fine-tune the reused layers for task B. After unfreezing the reused layers, it is usually a good idea to reduce the learning rate, once again to avoid damaging the reused weights

In [57]:
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4,
                           validation_data=(X_valid_B, y_valid_B))

Train on 200 samples, validate on 986 samples
Epoch 1/4
200/200 [==============================] - 0s 2ms/sample - loss: 0.5619 - accuracy: 0.6700 - val_loss: 0.5676 - val_accuracy: 0.6521
Epoch 2/4
200/200 [==============================] - 0s 280us/sample - loss: 0.5249 - accuracy: 0.7200 - val_loss: 0.5343 - val_accuracy: 0.6937
Epoch 3/4
200/200 [==============================] - 0s 247us/sample - loss: 0.4922 - accuracy: 0.7400 - val_loss: 0.5044 - val_accuracy: 0.7181
Epoch 4/4
200/200 [==============================] - 0s 280us/sample - loss: 0.4628 - accuracy: 0.7600 - val_loss: 0.4778 - val_accuracy: 0.7394


In [58]:
# unfreezing the layers


for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

In [59]:
# Recompiling is required after the unfreezing the layers

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(lr=1e-3),
                     metrics=["accuracy"])

In [60]:
# Retraining the model 

history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16,
                           validation_data=(X_valid_B, y_valid_B))

Train on 200 samples, validate on 986 samples
Epoch 1/16
200/200 [==============================] - 0s 2ms/sample - loss: 0.3865 - accuracy: 0.8250 - val_loss: 0.3362 - val_accuracy: 0.8682
Epoch 2/16
200/200 [==============================] - 0s 280us/sample - loss: 0.2701 - accuracy: 0.9300 - val_loss: 0.2611 - val_accuracy: 0.9280
Epoch 3/16
200/200 [==============================] - 0s 280us/sample - loss: 0.2081 - accuracy: 0.9650 - val_loss: 0.2151 - val_accuracy: 0.9513
Epoch 4/16
200/200 [==============================] - 0s 280us/sample - loss: 0.1694 - accuracy: 0.9800 - val_loss: 0.1841 - val_accuracy: 0.9635
Epoch 5/16
200/200 [==============================] - 0s 280us/sample - loss: 0.1427 - accuracy: 0.9800 - val_loss: 0.1602 - val_accuracy: 0.9716
Epoch 6/16
200/200 [==============================] - 0s 280us/sample - loss: 0.1219 - accuracy: 0.9850 - val_loss: 0.1424 - val_accuracy: 0.9797
Epoch 7/16
200/200 [==============================] - 0s 300us/sample - loss: 0.

You can see that, model performance has drastically increased.

---

In [61]:
# Evaluate on the test data for two model

# Evaluate for model B alone 
model_B.evaluate(X_test_B, y_test_B)



2000/2000 [==============================] - 0s 40us/sample - loss: 0.1426 - accuracy: 0.9695


[0.14263125371932983, 0.9695]

In [62]:

# Evaluation for the newly created model
model_B_on_A.evaluate(X_test_B, y_test_B)

2000/2000 [==============================] - 0s 40us/sample - loss: 0.0697 - accuracy: 0.9925


[0.06969365480542183, 0.9925]

In [63]:
# Error rate drop factor

(100 - 96.95) / (100 - 99.25)

4.066666666666663

---

# Faster Optimizers

### Momentum Optimization

In [64]:
optimizer = keras.optimizers.SGD(lr=0.001, momentum=0.9)

### Nesterov Accelerated Gradient

In [65]:
optimizer = keras.optimizers.SGD(lr=0.001, momentum=0.9, nesterov=True)

### AdaGrad

In [66]:
optimizer = keras.optimizers.Adagrad(lr=0.001)

### RMSProp

In [67]:
optimizer = keras.optimizers.RMSprop(lr=0.001, rho=0.9)

### Adam Optimization

In [68]:
optimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999)

### Nadam Optimization

In [69]:
optimizer = keras.optimizers.Nadam(lr=0.001, beta_1=0.9, beta_2=0.999)

---

# Avoiding Overfitting Through Regularization

---

### L1 and L2 Regularization

In [70]:
layer = keras.layers.Dense(100, activation="elu",
                           kernel_initializer="he_normal",
                           kernel_regularizer=keras.regularizers.l2(0.01))
# or l1(0.1) for ℓ1 regularization with a factor or 0.1
# or l1_l2(0.1, 0.01) for both ℓ1 and ℓ2 regularization, with factors 0.1 and 0.01 respectively

In [71]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="elu",
                       kernel_initializer="he_normal",
                       kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dense(100, activation="elu",
                       kernel_initializer="he_normal",
                       kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dense(10, activation="softmax",
                       kernel_regularizer=keras.regularizers.l2(0.01))
])

In [72]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])

In [73]:
n_epochs = 2

In [74]:
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

Train on 55000 samples, validate on 5000 samples
Epoch 1/2
55000/55000 [==============================] - 5s 96us/sample - loss: 1.6026 - accuracy: 0.8123 - val_loss: 0.7332 - val_accuracy: 0.8226
Epoch 2/2
55000/55000 [==============================] - 5s 84us/sample - loss: 0.7185 - accuracy: 0.8247 - val_loss: 0.6903 - val_accuracy: 0.8340


In [75]:
# This is the python func tool used in case if we require to repeat the 
 # things again and again.

from functools import partial

In [76]:
RegularizedDense = partial(keras.layers.Dense,
                           activation="elu",
                           kernel_initializer="he_normal",
                           kernel_regularizer=keras.regularizers.l2(0.01))

In [77]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    RegularizedDense(300),
    RegularizedDense(100),
    RegularizedDense(10, activation="softmax")
])

In [78]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])

In [79]:
n_epochs = 2

In [80]:
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

Train on 55000 samples, validate on 5000 samples
Epoch 1/2
55000/55000 [==============================] - 5s 94us/sample - loss: 1.6304 - accuracy: 0.8138 - val_loss: 0.7386 - val_accuracy: 0.8194
Epoch 2/2
55000/55000 [==============================] - 5s 84us/sample - loss: 0.7206 - accuracy: 0.8255 - val_loss: 0.6919 - val_accuracy: 0.8354


---

# Dropout Technique

In [81]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax")
])

In [82]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])

In [83]:
n_epochs = 2

In [84]:
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

Train on 55000 samples, validate on 5000 samples
Epoch 1/2
55000/55000 [==============================] - 6s 102us/sample - loss: 0.5730 - accuracy: 0.8030 - val_loss: 0.3922 - val_accuracy: 0.8592
Epoch 2/2
55000/55000 [==============================] - 5s 94us/sample - loss: 0.4249 - accuracy: 0.8440 - val_loss: 0.3383 - val_accuracy: 0.8740


---

# Alpha Dropout

In [85]:
tf.random.set_seed(42)
np.random.seed(42)

In [86]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.Dense(300, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax")
])

In [87]:
optimizer = keras.optimizers.SGD(lr=0.01, momentum=0.9, nesterov=True)

In [88]:
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [89]:
n_epochs = 20

In [90]:
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

Train on 55000 samples, validate on 5000 samples
Epoch 1/20
55000/55000 [==============================] - 5s 83us/sample - loss: 0.6616 - accuracy: 0.7616 - val_loss: 0.6689 - val_accuracy: 0.8258
Epoch 2/20
55000/55000 [==============================] - 4s 71us/sample - loss: 0.5527 - accuracy: 0.7969 - val_loss: 0.5835 - val_accuracy: 0.8388
Epoch 3/20
55000/55000 [==============================] - ETA: 0s - loss: 0.5260 - accuracy: 0.80 - 4s 69us/sample - loss: 0.5261 - accuracy: 0.8058 - val_loss: 0.5251 - val_accuracy: 0.8534
Epoch 4/20
55000/55000 [==============================] - 4s 71us/sample - loss: 0.5071 - accuracy: 0.8109 - val_loss: 0.4874 - val_accuracy: 0.8612
Epoch 5/20
55000/55000 [==============================] - 4s 72us/sample - loss: 0.4926 - accuracy: 0.8183 - val_loss: 0.4919 - val_accuracy: 0.8604
Epoch 6/20
55000/55000 [==============================] - 4s 71us/sample - loss: 0.4834 - accuracy: 0.8196 - val_loss: 0.5183 - val_accuracy: 0.8536
Epoch 7/20
5500

# END

---